In [1]:
%autosave 60

import tensorflow as tf
import numpy as np
from tensorflow.contrib.layers import batch_norm, dropout

Autosaving every 60 seconds


Exercise 8
===

Create a DQN to solve the "BipedalWalker-v2" gym.

In [2]:
import gym

env = gym.make("BipedalWalker-v2")
obs = env.reset()
print(obs)

[ 2.74735712e-03 -2.07262044e-05  1.61228981e-03 -1.59998643e-02
  9.18657258e-02 -2.12767534e-03  8.60322699e-01  2.97495971e-03
  1.00000000e+00  3.22708003e-02 -2.12752772e-03  8.53874087e-01
  1.50471103e-03  1.00000000e+00  4.40814108e-01  4.45820212e-01
  4.61422890e-01  4.89550292e-01  5.34102917e-01  6.02461159e-01
  7.09149063e-01  8.85932028e-01  1.00000000e+00  1.00000000e+00]


In [3]:
from matplotlib import pyplot as plt

%matplotlib inline

def render_env(env):
    plt.imshow(env.render(mode="rgb_array"))

#render_env(env)

In [4]:
print(env.action_space)
print(env.action_space.shape)
print(env.action_space.low)
print(env.action_space.high)

Box(4,)
(4,)
[-1 -1 -1 -1]
[1 1 1 1]


In [5]:
from tensorflow.contrib.layers import fully_connected

neurons_per_layer = [40]#[40, 40, 20]
learning_rate = 0.001 #0.001
n_discretized_actions = 3 # -1, 0, +1
n_actions = env.action_space.shape[0]
#n_outputs = env.action_space.shape[0]
n_outputs = n_discretized_actions ** n_actions
print("n_discretized_actions", n_discretized_actions, "n_actions", n_actions, "n_outputs", n_outputs)

def q_network(X_state, scope):
    current_layer = X_state
    layers = []
    with tf.variable_scope(scope) as scope:
        for neurons in neurons_per_layer:
            current_layer = fully_connected(current_layer, neurons)#, activation_fn=tf.nn.sigmoid)
            #print("current_layer shape", current_layer.get_shape())
        #print("input shape", current_layer.get_shape())
        # We use the tanh function because the output ranges from -1 to 1.
        outputs = fully_connected(current_layer, n_outputs, activation_fn=tf.nn.tanh)
    trainable_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope=scope.name)
    trainable_vars_by_name = {var.name[len(scope.name)]: var for var in trainable_vars}
    return outputs, trainable_vars_by_name

n_discretized_actions 3 n_actions 4 n_outputs 81


In [6]:
print(obs.shape)

(24,)


In [7]:
X_state = tf.placeholder(tf.float32, shape=[None, obs.shape[0]])
actor_q_values, actor_vars = q_network(X_state, "q_networks/actor")
critic_q_values, critic_vars = q_network(X_state, "q_networks/critic")

copy_ops = [actor_var.assign(critic_vars[var_name])
            for var_name, actor_var in actor_vars.items()]
copy_critic_to_actor = tf.group(*copy_ops)

In [8]:
X_action = tf.placeholder(tf.int32, shape=[None, env.action_space.shape[0]])
# This contains the Q-value for the actor's chosen action.
# q_value = tf.reduce_sum(critic_q_values * tf.one_hot(X_action, n_outputs), axis=1, keep_dims=True)
q_value = tf.reduce_sum(critic_q_values * tf.one_hot(n_outputs, 1), axis=1, keep_dims=True)
#q_value = critic_q_values
print("critic_q_values.get_shape()", critic_q_values.shape, "q_value.get_shape()", q_value.get_shape())

critic_q_values.get_shape() (?, 81) q_value.get_shape() (?, 1)


In [9]:
# The Actor's reward + future discounted estimated Q-value.
y = tf.placeholder(tf.float32, shape=[None, 1])
cost = tf.reduce_mean(tf.square(y - q_value))
global_step = tf.Variable(0, trainable=False, name="global_step")
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(cost, global_step=global_step)

init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [10]:
from collections import deque

replay_memory_size = 10000
replay_memory = deque([], maxlen=replay_memory_size)

def sample_memory(batch_size):
    indices = np.random.permutation(len(replay_memory))[:batch_size]
    cols = [[], [], [], [], []] # state, action, reward, next_state, continue
    for idx in indices:
        memory = replay_memory[idx]
        for col, value in zip(cols, memory):
            col.append(value)
    cols = [np.array(col) for col in cols]
    return (cols[0], cols[1], cols[2].reshape(-1, 1), cols[3], cols[4].reshape(-1, 1))


In [11]:
from itertools import product
discretized_actions = [-1, 0, 1]
action_space_universe = np.array(list(product(discretized_actions, discretized_actions, discretized_actions, discretized_actions)))

print(action_space_universe.shape)
#print(action_space_universe)

def action_index_to_action(action_index):
    return action_space_universe[action_index]

(81, 4)


In [ ]:
# epsilon greed algorithm to explore the state-action space of the environment at the beginning.
def epsilon_greedy(q_values, step):
    eps_min = 0.05 # At the start 5% of the time the actor will choose the greedy action.
    eps_max = 1.0 # Once it has finished exploring, the actor will choose the greedy action 100% of the time.
    eps_decay_steps = 100000
    
    epsilon = max(eps_min, eps_max - (eps_max - eps_min) * step / eps_decay_steps)
    if np.random.rand() < epsilon:
        #return np.random.uniform(env.action_space.low, env.action_space.high, size=n_actions)
        return np.random.randint(len(action_space_universe))
    else:
        #return q_values.reshape(n_outputs)
        return np.argmax(q_values) # the optimal/greedy action.

In [ ]:
import os

def preprocess_observation(obs):
    # I don't think I need this method.
    return obs

n_steps = 110000
# Start training the critic DQN after 1000 game iterations.
# This has to be a lot bigger than the batch_size defined below.
training_start = 1000
training_interval = 3 # Run a training step every 3 game iterations start training_start.
save_steps = 50
copy_steps = 25 # Copy the critic to the actor every 25 training steps.
discount_rate = 0.95 # 0.95
batch_size = 50
iteration = 0
checkpoint_path = "./BipedalWalker-v2.ckpt"
done = True # Environment needs to be reset
total_reward = 0
current_steps = 0

with tf.Session() as sess:
    if os.path.isfile(checkpoint_path):
        print("Restoring checkpoint")
        saver.restore(sess, checkpoint_path)
    else:
        init.run()
    while True:
        step = global_step.eval()
        if step >= n_steps:
            break
        elif step != 0and step % 10000 == 0:
            print("global_step", step)
        iteration += 1
        if done:
            obs = env.reset()
            state = preprocess_observation(obs)
            total_reward = 0
            current_steps = 0
        
        # Actor evaluates what to do
        q_values = actor_q_values.eval(feed_dict={X_state: [state]})
        #print(q_values)
        action_index = epsilon_greedy(q_values, step)
        action = action_index_to_action(action_index)
        #print(action)
        
        #render_env(env)
        # Actor takes action
        obs, reward, done, info = env.step(action)
        if done:
            print("Agent died: current_steps", current_steps, "total_reward", total_reward)
            render_env(env)
        elif (150 < current_steps and current_steps % 10 == 0):
            render_env(env)
        total_reward += reward
        current_steps += 1
        next_state = preprocess_observation(obs)
        
        # Memorise action.
        replay_memory.append((state, action, reward, next_state, 1.0 - done))
        state = next_state
        
        if iteration < training_start or iteration % training_interval != 0:
            continue
        
        # Train the critic.
        X_state_val, X_action_val, rewards, X_next_state_val, continues = (
            sample_memory(batch_size))
        next_q_values = actor_q_values.eval(feed_dict={X_state: X_next_state_val})
        max_next_q_values = np.max(next_q_values, axis=1, keepdims=True)
        # Calculate the Actor's reward + future discounted estimated Q-value.
        #print("rewards", rewards.shape, "continues", continues.shape, "max_next_q_values", max_next_q_values.shape)
        y_val = rewards + continues * discount_rate * max_next_q_values
        #print(X_state, X_action, y)
        #print("X_state_val", X_state_val.shape, "X_action", X_action_val.shape, "y_val", y_val.shape)
        training_op.run(feed_dict={X_state: X_state_val,
                                  X_action: X_action_val,
                                  y: y_val})
        
        if step % copy_steps == 0:
            copy_critic_to_actor.run()
        
        if step % save_steps == 0:
            saver.save(sess, checkpoint_path)

Agent died: current_steps 88 total_reward -15.973937595615778
Agent died: current_steps 72 total_reward -13.45541904788898
Agent died: current_steps 53 total_reward -4.201687112902602
Agent died: current_steps 59 total_reward -2.7168254013545843
Agent died: current_steps 137 total_reward -14.938071741284773
Agent died: current_steps 72 total_reward 0.08735586924540233
Agent died: current_steps 1599 total_reward -122.87969524368668
Agent died: current_steps 213 total_reward -26.68893238030558
Agent died: current_steps 1599 total_reward -119.85622503653025
Agent died: current_steps 52 total_reward -14.391935125932097
Agent died: current_steps 71 total_reward -7.664384290754172
Agent died: current_steps 1599 total_reward -94.91008623886334
Agent died: current_steps 91 total_reward -20.73060168629947
Agent died: current_steps 1599 total_reward -108.66638557585858
Agent died: current_steps 49 total_reward -7.636111337793987
Agent died: current_steps 1599 total_reward -130.41477990458156
Age

Agent died: current_steps 1599 total_reward -108.14392995958421
Agent died: current_steps 61 total_reward -12.955021409107363
Agent died: current_steps 1599 total_reward -103.8742836629783
Agent died: current_steps 84 total_reward -3.8199603649936598
Agent died: current_steps 1599 total_reward -133.94618840732545
Agent died: current_steps 220 total_reward -38.22319666688885
Agent died: current_steps 77 total_reward -12.654846983921402
Agent died: current_steps 44 total_reward -12.848543182131522
Agent died: current_steps 66 total_reward -19.5180803723118
Agent died: current_steps 1599 total_reward -120.39897695316584
Agent died: current_steps 57 total_reward -13.711500725942354
Agent died: current_steps 65 total_reward -17.92399236278484
Agent died: current_steps 1599 total_reward -122.34376900870085
Agent died: current_steps 1599 total_reward -122.09429701803217
Agent died: current_steps 63 total_reward -2.327600962860512
Agent died: current_steps 80 total_reward -2.040906349827226
Ag

Agent died: current_steps 68 total_reward -16.03703583888958
Agent died: current_steps 334 total_reward -10.14673841541504
Agent died: current_steps 1599 total_reward -87.04916050532623
Agent died: current_steps 1599 total_reward -87.00842457813675
Agent died: current_steps 1345 total_reward -80.41504716888464
Agent died: current_steps 71 total_reward -16.331749296011395
Agent died: current_steps 1599 total_reward -81.01149494732157
Agent died: current_steps 917 total_reward -40.00318152027601
Agent died: current_steps 574 total_reward -23.822030239215625
Agent died: current_steps 1309 total_reward -51.78304150195118
Agent died: current_steps 622 total_reward -36.95572718939063
global_step 70000
global_step 70000
global_step 70000
Agent died: current_steps 1599 total_reward -93.72269632899726
Agent died: current_steps 511 total_reward -34.36231517360293
Agent died: current_steps 141 total_reward -4.933669305431325
Agent died: current_steps 888 total_reward -61.30025427493161
Agent died

In [ ]:
print("Done")

For:
neurons_per_layer = [40, 10]
learning_rate = 0.001

The RL agent learnt how to get an average reward of 3 +- 2. I assume it is learning how to fall over. This is good progress. Running the agent with these hyper paremeters again, did not produce these same results unfortunately.

Rendering the environment has shown that a reward of 3-5 is probably just the agent falling as far forward as possible. Dumb robot.

After rendering the environment every 50 frames, I can see that it either falls over quickly or it splits it legs and very slowly inches forward and then gets killed after 1599 steps.

I couldn't get the agent to learn how to walk and when I checked my code, I realised that I was using the Q-values from the agent as the torque values for it's action, which basically made it stateless (I think).